# Proyecto Cardio Good Fitness Case Study

Este data set es el resultado de un estudio de mercado de AdRigh al cual le asignaron la tarea de identificar el perfil del cliente típico para cada tipo de caminadora ofrecido por CardioGood Fitness.
Los datos hace parte de las personas que compraron una caminadora tres meses antes en una tiene minorista de CardioGood Fitness.
Las variables del data set son: 
- Product: Producto comprado TM195, TM498 o TM798
- Age: Edad en años
- Gender: Género
- Education: Educación en años
- MaritalStatus: Estado civil, soltero o en pareja
- Usuge: Número promedio de veces que el cliente planea usar la caminadora cada semana
- Fitness: Autoevaluación del estado físico en una escala del 1 al 5, donde 1 es mala forma física y 5 es excelente forma física
- Income: Ingreso familiar anual ($)
- Miles: Número promedio de millas que el cliente espera caminar/correr cada semana

https://www.kaggle.com/datasets/saurav9786/cardiogoodfitness?select=CardioGoodFitness.csv

In [2]:
# importación de librerias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Dataset Information

In [6]:
data = pd.read_csv("CardioGoodFitness.csv")
data.head

<bound method NDFrame.head of     Product  Age  Gender  Education MaritalStatus  Usage  Fitness  Income  \
0     TM195   18    Male         14        Single      3        4   29562   
1     TM195   19    Male         15        Single      2        3   31836   
2     TM195   19  Female         14     Partnered      4        3   30699   
3     TM195   19    Male         12        Single      3        3   32973   
4     TM195   20    Male         13     Partnered      4        2   35247   
..      ...  ...     ...        ...           ...    ...      ...     ...   
175   TM798   40    Male         21        Single      6        5   83416   
176   TM798   42    Male         18        Single      5        4   89641   
177   TM798   45    Male         16        Single      5        5   90886   
178   TM798   47    Male         18     Partnered      4        5  104581   
179   TM798   48    Male         18     Partnered      4        5   95508   

     Miles  
0      112  
1       75  
2     

In [7]:
# tamaño del archivo
data.shape

(180, 9)

In [8]:
# nombre de las columnas
data.columns

Index(['Product', 'Age', 'Gender', 'Education', 'MaritalStatus', 'Usage',
       'Fitness', 'Income', 'Miles'],
      dtype='object')

In [9]:
# información de cada tipo de dato
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product        180 non-null    object
 1   Age            180 non-null    int64 
 2   Gender         180 non-null    object
 3   Education      180 non-null    int64 
 4   MaritalStatus  180 non-null    object
 5   Usage          180 non-null    int64 
 6   Fitness        180 non-null    int64 
 7   Income         180 non-null    int64 
 8   Miles          180 non-null    int64 
dtypes: int64(6), object(3)
memory usage: 12.8+ KB


In [10]:
# revisar nulos 
data.isnull().sum()

Product          0
Age              0
Gender           0
Education        0
MaritalStatus    0
Usage            0
Fitness          0
Income           0
Miles            0
dtype: int64

In [11]:
# principales estadisticas
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,180.0,28.788889,6.943498,18.0,24.00,26.0,33.00,50.0
Education,180.0,15.572222,1.617055,12.0,14.00,16.0,16.00,21.0
Usage,180.0,3.455556,1.084797,2.0,3.00,3.0,4.00,7.0
Fitness,180.0,3.311111,0.958869,1.0,3.00,3.0,4.00,5.0
Income,180.0,53719.577778,16506.684226,29562.0,44058.75,50596.5,58668.00,104581.0
Miles,180.0,103.194444,51.863605,21.0,66.00,94.0,114.75,360.0


In [12]:
# valores unicos de cada variable categorica
data.nunique()

Product           3
Age              32
Gender            2
Education         8
MaritalStatus     2
Usage             6
Fitness           5
Income           62
Miles            37
dtype: int64